# Dependencies and Setup

In [105]:
import pandas as pd
import csv
from geopy.geocoders import Nominatim
import folium 

# EXTRACT

In [106]:
# File to Load
crime_to_load = "Resources/Baltimore_2019_Crime_Only.csv"
cctv_to_load = "Resources/CCTV_hoods.csv"

from sqlalchemy import create_engine

In [107]:
# Read the CSV files
crime_df = pd.read_csv(crime_to_load)
cctv_df = pd.read_csv(cctv_to_load)

In [108]:
crime_df.head(10)

,CrimeDate,CrimeTime,CrimeCode,Location,Description,Weapon,District,Neighborhood,Total Incidents
0,07/06/2019,01:30:00,5B,0 HILLSIDE RD,BURGLARY,NaN,NORTHERN,ROLAND PARK,1
1,07/06/2019,09:00:00,7A,1600 APPLETON ST,AUTO THEFT,NaN,WESTERN,EASTERWOOD,1
2,07/06/2019,07:03:00,6E,2500 EDGECOMBE CIR N,LARCENY,NaN,NORTHERN,PARKLANE,1
3,07/06/2019,02:30:00,3AK,PL & N HIGHLAND AV,ROBBERY - STREET,KNIFE,SOUTHEAST,NaN,1
4,07/06/2019,10:05:00,6C,700 WASHINGTON BLVD,LARCENY,NaN,SOUTHERN,WASHINGTON VILLAGE,1
5,07/06/2019,19:00:00,4B,2100 32ND ST,AGG. ASSAULT,KNIFE,NORTHEAST,NaN,1
6,07/06/2019,17:31:00,4C,3000 GLENMORE AVE,AGG. ASSAULT,OTHER,NORTHEAST,NaN,1
7,07/06/2019,16:38:00,4C,3700 2ND ST,AGG. ASSAULT,OTHER,SOUTHERN,BROOKLYN,1
8,07/06/2019,15:21:00,4E,1700 RUSSELL ST,COMMON ASSAULT,NaN,SOUTHERN,CARROLL - CAMDEN INDUSTRIAL AREA,1
9,07/06/2019,15:40:00,5A,3800 5TH ST,BURGLARY,NaN,SOUTHERN,BROOKLYN,1


In [114]:
sample = crime_df.sample(10)
sample

,CrimeDate,CrimeTime,CrimeCode,Location,Description,Weapon,District,Neighborhood,Total Incidents
9861,04/19/2019,11:59:00,4A,2500 E OLIVER ST,AGG. ASSAULT,FIREARM,EASTERN,BEREA,1
10308,04/16/2019,13:00:00,6F,100 N CALVERT ST,LARCENY,NaN,CENTRAL,DOWNTOWN,1
4379,06/01/2019,19:48:00,4A,200 S LOUDON AVE,AGG. ASSAULT,FIREARM,SOUTHWEST,IRVINGTON,1
1670,06/22/2019,16:20:00,4E,5200 YORK RD,COMMON ASSAULT,NaN,NORTHERN,LOYOLA/NOTRE DAME,1
22174,01/01/2019,10:35:00,3B,3300 WILKENS AVE,ROBBERY - STREET,NaN,SOUTHWEST,SAINT AGNES,1
22158,01/01/2019,20:45:00,3JF,1900 CECIL AVE,ROBBERY - RESIDENCE,FIREARM,EASTERN,EAST BALTIMORE MIDWA,1
1294,06/25/2019,19:00:00,6D,600 QUEENSGATE RD,LARCENY FROM AUTO,NaN,SOUTHWEST,BEECHFIELD,1
13603,03/19/2019,13:45:00,4B,RD & WOODHOME AV,AGG. ASSAULT,KNIFE,NORTHEAST,NaN,1
11667,04/04/2019,10:31:00,9S,1700 GERTRUDE ST,SHOOTING,FIREARM,SOUTHWEST,NORTHWEST COMMUNITY,1
12393,03/29/2019,17:15:00,4A,3600 DUVALL AVE,AGG. ASSAULT,FIREARM,NORTHWEST,MEDFORD,1


In [115]:
cctv_df.head()

,cameraLocation,cameraNumber,cameraProject,Location 1,latitude,longitude,neighborhood
0,Eutaw and Lexington Market,1,Downtown,"(39.290996, -76.621073999999993)",39.290996,-76.621074,NorthEutawStreet
1,Eutaw and Fayette,2,Downtown,"(39.29048796517317, -76.623665143680341)",39.290488,-76.623665,RidgleysDelight
2,Eutaw and Baltimore,3,Downtown,"(39.289324000000001, -76.620985000000005)",39.289324,-76.620985,WestBaltimoreStreet
3,Eutaw and Redwood,4,Downtown,"(39.288677999999997, -76.620947999999999)",39.288678,-76.620948,RidgleysDelight
4,Eutaw and Lombard,5,Downtown,"(39.287528000000002, -76.620853999999994)",39.287528,-76.620854,RidgleysDelight


# Create database connection


In [116]:
connection_string = "postgres:postgres@localhost:5432/postgres"
engine = create_engine(f'postgresql://{connection_string}')

# Confirm tables
engine.table_names()

['crime', 'cctv']

# Confirm tables

In [117]:
crime_df.to_sql(name='crime', con=engine, if_exists='append', index=True)

In [118]:
cctv_df.to_sql(name='cctv', con=engine, if_exists='append', index=True)

# TRANSFORM

In [119]:
# Changing "Location 1" form cctv_df to 'latitude' and 'longitiude'
# This is required for geocoding which we want to use to find addresses of deployed CCTV

# Create two lists for the loop results to be placed
lat = []
lon = []

# For each row in a varible,
for row in cctv_df['Location 1']:
    # Try to,
    try:
        #Remove ( from the string
        row = row.replace("(", "")
        row = row.replace(")", "")
        # Split the row by comma, convert to float, and append
        # everything before the comma to lat
        lat.append(float(row.split(',')[0]))
        # Split the row by comma, convert to float, and append
        # everything after the comma to lon
        lon.append(float(row.split(',')[1]))
    # But if you get an error
    except:
        # append a missing value to lat
        lat.append(np.NaN)
        # append a missing value to lon
        lon.append(np.NaN)

# Create two new columns from lat and lon
cctv_df['latitude'] = lat
cctv_df['longitude'] = lon

In [120]:
cctv_df.head()

,cameraLocation,cameraNumber,cameraProject,Location 1,latitude,longitude,neighborhood
0,Eutaw and Lexington Market,1,Downtown,"(39.290996, -76.621073999999993)",39.290996,-76.621074,NorthEutawStreet
1,Eutaw and Fayette,2,Downtown,"(39.29048796517317, -76.623665143680341)",39.290488,-76.623665,RidgleysDelight
2,Eutaw and Baltimore,3,Downtown,"(39.289324000000001, -76.620985000000005)",39.289324,-76.620985,WestBaltimoreStreet
3,Eutaw and Redwood,4,Downtown,"(39.288677999999997, -76.620947999999999)",39.288678,-76.620948,RidgleysDelight
4,Eutaw and Lombard,5,Downtown,"(39.287528000000002, -76.620853999999994)",39.287528,-76.620854,RidgleysDelight


In [121]:
print(location.address)

University of Maryland Baltimore, West Baltimore Street, Ridgleys Delight, Baltimore, Maryland, 21223, USA


In [104]:
folium_map = folium.Map(location=[39.29, -76.61],
                        zoom_start=13)
                        #tiles="CartoDB dark_matter")

for cameraNumber in cctv_df:
 #radius of circles
    # choose the color of the marker
    if int(cctv_df.loc[key, "cameraNumber"])<2:
        color="#39cc28" # green
    elif int(cctv_df.loc[key, "cameraNumber"])>=2:
        color="#2743cc" #blue
    else:
        color="#ea1f09" # green
    radius = int(cctv_df.loc[key, "cameraNumber"])
    marker = folium.CircleMarker(location=cctv_df[key],
                                 radius=radius,
                                 color=color).add_to(folium_map)

folium_map

KeyError: 'the label [cameraLocation] is not in the [index]'

In [ ]:
map_osm = folium.Map(location=[39.29, -76.61], zoom_start=11)
map_osm